## 9.1 함수 감싸기
- 함수에 추가적인 처리를 하는 래퍼 레이어(wrapper layer)를 넣고 싶음

In [1]:
#데코레이터 함수 정의
import time
from functools import wraps

def timethis(func):
    '''
    실행 시간 보고하는 데코레이터
    '''
    @wraps(func)
    def wrapper(*args, **kwargs): #어떤 인자라도 받을 수 있도록 *args, **kwargs 사용
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

In [2]:
@timethis
def countdown(n):
    '''
    Counts down
    '''
    while n > 0 :
        n -= 1

In [3]:
countdown(100000)

countdown 0.007357120513916016


In [5]:
countdown(10000000)

countdown 0.3835785388946533


## 9.5 사용자가 조절 가능한 속성을 가진 데코레이터 정의

In [6]:
from functools import wraps, partial
import logging

def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj)
    setattr(obj, func.__name__, func)
    return func

In [10]:
def logged(level, name=None, message=None):
    '''
    함수에 로깅 추가. level은 로깅 레벨
    name : 로거 이름
    message : 로그 메세지
    name과 message가 명시되지 않으면 모듈 이름을 기본 값으로 함
    '''
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
    
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)

        #세터 함수 첨부
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel

        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal logmsg
            logmsg = newmsg

        return wrapper
    return decorate

In [12]:
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

In [13]:
import logging
logging.basicConfig(level=logging.DEBUG)
add(2, 3)

DEBUG:__main__:add


5

In [14]:
# change the log message
add.set_message('Add called')
add(2, 3)

DEBUG:__main__:Add called


5

In [15]:
#change the log message
add.set_level(logging.WARNING)
add(2, 3)

5

In [24]:
@timethis
@logged(logging.DEBUG)
def countdown(n):
    while n>0 :
        n -= 1

In [25]:
countdown(10000)

DEBUG:__main__:countdown


countdown 0.001781463623046875


In [27]:
countdown.set_level(logging.WARNING)
countdown.set_message("counting down to zero")
countdown(10000)

countdown 0.0023946762084960938


## 9.9 클래스 데코레이터 정의

In [16]:
import types
from functools import wraps

class Profiled:
    def __init__(self, func):
        wraps(func)(self)
        self.ncalls = 0
        
    def __call__(self, *args, **kwargs):
        self.ncalls += 1
        return self.__wrapped__(*args, **kwargs)
    
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return types.MethodType(self, instance)

In [17]:
@Profiled
def add(x, y):
    return x + y

class Spam:
    @Profiled
    def bar(self, x):
        print(self, x)

In [18]:
add(2, 3)

5

In [19]:
add(4, 5)

9

In [20]:
add.ncalls

2

In [21]:
s = Spam()
s.bar(1)

<__main__.Spam object at 0x7f2a54d94208> 1


In [22]:
s.bar(2)

<__main__.Spam object at 0x7f2a54d94208> 2


In [23]:
Spam.bar.ncalls

2